# Notebook 05: Síntesis de Resultados

Este notebook consolida todos los análisis realizados:
1. Resumen de hallazgos principales
2. Visualizaciones finales de calidad publicación
3. Exportación de resultados para la aplicación web

In [ ]:
# Importar librerías
import sys
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from pathlib import Path
import warnings
import json

warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

print("Librerías cargadas")

In [ ]:
# Rutas
DATA_PATH = Path('../data/raw/isla_de_pascua')
PROCESSED_PATH = Path('../data/processed')
OUTPUT_PATH = Path('../outputs')
FIGURES_PATH = OUTPUT_PATH / 'figures'
REPORTS_PATH = OUTPUT_PATH / 'reports'

# Crear directorios si no existen
FIGURES_PATH.mkdir(parents=True, exist_ok=True)
REPORTS_PATH.mkdir(parents=True, exist_ok=True)

## 1. Cargar Todos los Datos Procesados

In [ ]:
# Cargar datos base
buildings = gpd.read_file(DATA_PATH / 'isla_de_pascua_buildings.geojson')
boundary = gpd.read_file(DATA_PATH / 'isla_de_pascua_boundary.geojson')
streets = gpd.read_file(DATA_PATH / 'isla_de_pascua_streets.geojson')
amenities = gpd.read_file(DATA_PATH / 'isla_de_pascua_amenities.geojson')

# Reproyectar a UTM
buildings_utm = buildings.to_crs(epsg=32712)
boundary_utm = boundary.to_crs(epsg=32712)
streets_utm = streets.to_crs(epsg=32712)
amenities_utm = amenities.to_crs(epsg=32712)

# Calcular métricas
buildings_utm['area_m2'] = buildings_utm.geometry.area

print("Datos cargados:")
print(f"  - Edificios: {len(buildings_utm)}")
print(f"  - Calles: {len(streets_utm)}")
print(f"  - Amenidades: {len(amenities_utm)}")

## 2. Resumen Estadístico del Territorio

In [ ]:
# Calcular estadísticas territoriales
area_total_km2 = boundary_utm.geometry.area.sum() / 1e6
area_construida_m2 = buildings_utm['area_m2'].sum()
area_construida_km2 = area_construida_m2 / 1e6
densidad_edificios = len(buildings_utm) / area_total_km2

stats_territorio = {
    'Área Total (km²)': area_total_km2,
    'Área Construida (km²)': area_construida_km2,
    'Porcentaje Construido (%)': (area_construida_km2 / area_total_km2) * 100,
    'Número de Edificios': len(buildings_utm),
    'Densidad (edificios/km²)': densidad_edificios,
    'Área Media por Edificio (m²)': buildings_utm['area_m2'].mean(),
    'Longitud Total de Calles (km)': streets_utm.geometry.length.sum() / 1000,
    'Número de Amenidades': len(amenities_utm)
}

print("="*60)
print("ESTADÍSTICAS TERRITORIALES - ISLA DE PASCUA")
print("="*60)
for key, value in stats_territorio.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Guardar estadísticas como JSON para la app web
stats_json = {k: float(v) if isinstance(v, (int, float, np.number)) else v 
              for k, v in stats_territorio.items()}

with open(REPORTS_PATH / 'territorio_stats.json', 'w', encoding='utf-8') as f:
    json.dump(stats_json, f, indent=2, ensure_ascii=False)

print(f"Estadísticas guardadas en {REPORTS_PATH / 'territorio_stats.json'}")

## 3. Mapa General del Territorio

In [ ]:
# Mapa general de alta calidad
fig, ax = plt.subplots(figsize=(16, 12))

# Base: Límite del territorio
boundary_utm.plot(ax=ax, facecolor='#f5f5f5', edgecolor='#333333', linewidth=2)

# Calles
streets_utm.plot(ax=ax, color='#666666', linewidth=0.5, alpha=0.5)

# Edificios coloreados por área
buildings_utm.plot(ax=ax, column='area_m2', cmap='YlOrRd', 
                   legend=True, legend_kwds={'label': 'Área del Edificio (m²)',
                                            'shrink': 0.6})

# Amenidades
amenities_point = amenities_utm[amenities_utm.geometry.geom_type == 'Point']
if len(amenities_point) > 0:
    amenities_point.plot(ax=ax, color='blue', markersize=50, marker='o', 
                         alpha=0.7, label='Amenidades')

# Títulos y etiquetas
ax.set_title('Mapa General del Territorio - Isla de Pascua', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Este (m)', fontsize=12)
ax.set_ylabel('Norte (m)', fontsize=12)

# Escala aproximada
from matplotlib.patches import FancyBboxPatch
scalebar_x = ax.get_xlim()[0] + (ax.get_xlim()[1] - ax.get_xlim()[0]) * 0.05
scalebar_y = ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.05
ax.plot([scalebar_x, scalebar_x + 1000], [scalebar_y, scalebar_y], 'k-', linewidth=3)
ax.text(scalebar_x + 500, scalebar_y + 100, '1 km', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_PATH / 'mapa_general_territorio.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Mapa guardado en {FIGURES_PATH / 'mapa_general_territorio.png'}")

## 4. Resumen de Hallazgos

In [ ]:
# Crear resumen de hallazgos
hallazgos = """
# RESUMEN DE HALLAZGOS - ANÁLISIS TERRITORIAL ISLA DE PASCUA

## 1. Caracterización del Territorio

La Isla de Pascua presenta un patrón de desarrollo urbano concentrado 
principalmente en torno a Hanga Roa, con expansión hacia zonas periféricas.

### Métricas Clave:
- Área total analizada: {area:.2f} km²
- Total de edificaciones: {n_edificios:,}
- Densidad promedio: {densidad:.1f} edificios/km²
- Área construida promedio: {area_media:.1f} m²/edificio

## 2. Análisis de Autocorrelación Espacial

El análisis ESDA reveló patrones significativos de clustering espacial:
- Se identificaron zonas de alta densidad (Hot Spots) en el centro urbano
- Zonas de baja densidad (Cold Spots) en áreas periféricas y naturales
- La autocorrelación espacial es estadísticamente significativa

## 3. Geoestadística

El análisis geoestadístico permitió:
- Modelar la variabilidad espacial de la densidad de edificios
- Comparar métodos de interpolación (Kriging vs IDW)
- Cuantificar la incertidumbre en las predicciones espaciales

## 4. Machine Learning Espacial

Los modelos predictivos identificaron las siguientes variables como 
principales determinantes de la densidad urbana:
1. Distancia al centro urbano
2. Densidad de amenidades cercanas
3. Conectividad vial (longitud de calles)
4. Características de los vecinos espaciales

## 5. Recomendaciones

1. **Planificación Urbana**: Considerar los patrones de clustering para
   la zonificación y planificación de servicios.

2. **Conservación**: Proteger las zonas de baja densidad que coinciden
   con áreas de valor patrimonial y natural.

3. **Infraestructura**: Priorizar inversión en conectividad vial en
   zonas con alto potencial de desarrollo.
""".format(
    area=area_total_km2,
    n_edificios=len(buildings_utm),
    densidad=densidad_edificios,
    area_media=buildings_utm['area_m2'].mean()
)

# Guardar resumen
with open(REPORTS_PATH / 'resumen_hallazgos.md', 'w', encoding='utf-8') as f:
    f.write(hallazgos)

print(hallazgos)

## 5. Dashboard de Métricas

In [ ]:
# Crear figura tipo dashboard
fig = plt.figure(figsize=(20, 12))

# Grid de subplots
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# 1. Mapa de ubicación (grande)
ax1 = fig.add_subplot(gs[0, 0:2])
boundary_utm.plot(ax=ax1, facecolor='lightblue', edgecolor='navy', linewidth=2)
buildings_utm.plot(ax=ax1, color='coral', alpha=0.6, markersize=1)
ax1.set_title('Distribución de Edificaciones', fontsize=14, fontweight='bold')
ax1.set_axis_off()

# 2. Distribución de tamaños
ax2 = fig.add_subplot(gs[0, 2])
buildings_utm['area_m2'].clip(upper=500).hist(ax=ax2, bins=30, color='steelblue', 
                                               edgecolor='white', alpha=0.8)
ax2.set_xlabel('Área (m²)')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Áreas', fontsize=14, fontweight='bold')
ax2.axvline(buildings_utm['area_m2'].median(), color='red', linestyle='--', 
            label=f'Mediana: {buildings_utm["area_m2"].median():.0f} m²')
ax2.legend()

# 3. Tipos de amenidades
ax3 = fig.add_subplot(gs[1, 0])
if 'amenity' in amenities_utm.columns:
    amenity_counts = amenities_utm['amenity'].value_counts().head(10)
    amenity_counts.plot(kind='barh', ax=ax3, color='teal', alpha=0.8)
    ax3.set_xlabel('Cantidad')
    ax3.set_title('Top 10 Tipos de Amenidades', fontsize=14, fontweight='bold')
else:
    ax3.text(0.5, 0.5, 'Sin datos de amenidades', ha='center', va='center',
             fontsize=12, transform=ax3.transAxes)
    ax3.set_title('Tipos de Amenidades', fontsize=14, fontweight='bold')

# 4. Métricas clave
ax4 = fig.add_subplot(gs[1, 1])
ax4.axis('off')

metrics_text = f"""
MÉTRICAS CLAVE
{'─'*30}

📍 Área total: {area_total_km2:.2f} km²

🏠 Edificios: {len(buildings_utm):,}

📊 Densidad: {densidad_edificios:.1f} edif/km²

🛣️ Calles: {streets_utm.geometry.length.sum()/1000:.1f} km

🏪 Amenidades: {len(amenities_utm)}
"""
ax4.text(0.1, 0.9, metrics_text, transform=ax4.transAxes, fontsize=14,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 5. Boxplots de áreas por tipo
ax5 = fig.add_subplot(gs[1, 2])
# Categorizar edificios por tamaño
buildings_utm['size_cat'] = pd.cut(buildings_utm['area_m2'], 
                                   bins=[0, 50, 100, 200, 500, float('inf')],
                                   labels=['<50m²', '50-100m²', '100-200m²', '200-500m²', '>500m²'])
size_counts = buildings_utm['size_cat'].value_counts().sort_index()
size_counts.plot(kind='bar', ax=ax5, color='coral', alpha=0.8, edgecolor='white')
ax5.set_xlabel('Categoría de Tamaño')
ax5.set_ylabel('Número de Edificios')
ax5.set_title('Edificios por Categoría', fontsize=14, fontweight='bold')
ax5.tick_params(axis='x', rotation=45)

plt.suptitle('ANÁLISIS TERRITORIAL ISLA DE PASCUA', fontsize=20, fontweight='bold', y=1.02)
plt.savefig(FIGURES_PATH / 'dashboard_resumen.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Dashboard guardado en {FIGURES_PATH / 'dashboard_resumen.png'}")

## 6. Exportar Datos para la Aplicación Web

In [ ]:
# Preparar datos simplificados para la web
# Simplificar geometrías para mejor rendimiento
boundary_simple = boundary.copy()
boundary_simple['geometry'] = boundary_simple.geometry.simplify(tolerance=0.0001)

buildings_simple = buildings.copy()
buildings_simple['geometry'] = buildings_simple.geometry.simplify(tolerance=0.00005)
buildings_simple['area_m2'] = buildings_utm['area_m2'].values

# Guardar como GeoJSON para uso web
boundary_simple.to_file(PROCESSED_PATH / 'boundary_web.geojson', driver='GeoJSON')
buildings_simple[['geometry', 'area_m2']].to_file(PROCESSED_PATH / 'buildings_web.geojson', driver='GeoJSON')

print("Datos exportados para la web:")
print(f"  - {PROCESSED_PATH / 'boundary_web.geojson'}")
print(f"  - {PROCESSED_PATH / 'buildings_web.geojson'}")

In [ ]:
# Crear resumen de outputs generados
outputs_summary = {
    'figures': [
        'mapa_general_territorio.png',
        'dashboard_resumen.png',
        'variogram_experimental.png',
        'variogram_models_fit.png',
        'interpolation_comparison.png',
        'cross_validation_results.png'
    ],
    'reports': [
        'territorio_stats.json',
        'resumen_hallazgos.md',
        'cross_validation_summary.csv'
    ],
    'processed_data': [
        'boundary_web.geojson',
        'buildings_web.geojson',
        'interpolation_results.gpkg'
    ]
}

with open(REPORTS_PATH / 'outputs_manifest.json', 'w') as f:
    json.dump(outputs_summary, f, indent=2)

print("\n" + "="*60)
print("RESUMEN DE OUTPUTS GENERADOS")
print("="*60)
for category, files in outputs_summary.items():
    print(f"\n{category.upper()}:")
    for f in files:
        print(f"  - {f}")

## 7. Conclusiones Finales

In [ ]:
print("""
═══════════════════════════════════════════════════════════════════════════════
                    ANÁLISIS TERRITORIAL COMPLETADO
═══════════════════════════════════════════════════════════════════════════════

Este laboratorio integrador analizó la Isla de Pascua utilizando:

✅ ANÁLISIS EXPLORATORIO (ESDA)
   - Estadísticas descriptivas espaciales
   - Mapas temáticos de densidad y distribución
   - Identificación de patrones espaciales

✅ AUTOCORRELACIÓN ESPACIAL
   - Índice de Moran I global
   - Análisis LISA (Local Moran)
   - Hot Spots con Getis-Ord Gi*

✅ GEOESTADÍSTICA
   - Semivariogramas experimentales
   - Ajuste de modelos teóricos
   - Interpolación Kriging vs IDW
   - Validación cruzada leave-one-out

✅ MACHINE LEARNING ESPACIAL
   - Feature engineering geoespacial
   - Modelos Random Forest y XGBoost
   - Validación espacial con GroupKFold
   - Interpretabilidad con SHAP

✅ APLICACIÓN WEB
   - Dashboard interactivo con Streamlit
   - Mapas con Folium
   - Exportación de resultados

═══════════════════════════════════════════════════════════════════════════════
""")